In [1]:
import numpy as np
import pandas as pd
from biom import (load_table,
                  Table)
from qiime2 import (Artifact,
                    Metadata, Visualization)
from skbio import OrdinationResults, DistanceMatrix
from skbio.stats.distance import permanova
from qiime2.plugins.diversity.actions import beta, pcoa
from qiime2.plugins.gemelli.actions import rpca
from tqdm.notebook import tqdm
from scipy.spatial import ConvexHull

import colorsys
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mc
from matplotlib.pyplot import cm
from matplotlib.colors import to_hex
plt.style.use('seaborn') 

%matplotlib inline

In [2]:
# import raw data
mf_df = pd.read_csv('../data/tumor_vs_nat/t_vs_nat_scaled_metadata_TCGA.csv', index_col=0)
mf_df = mf_df[mf_df.experimental_strategy == 'WGS']
tbl_df = pd.read_csv('../data/tumor_vs_nat/t_vs_nat_scaled_rel_abundances_TCGA.csv', index_col=0)
mf_df = mf_df.reindex(tbl_df.index)

mf_df.head(5)


,sample_name,run_prefix,experimental_strategy,cgc_base_name,filename,analyte_amount,analyte_A260A280Ratio,aliquot_concentration,cgc_id,cgc_filename,...,tissue_source_site_label,country_of_sample_procurement,portion_is_ffpe,pathologic_t_label,pathologic_n_label,histological_diagnosis_label,pathologic_stage_label,PlateCenter,PlateCenterFlag,disease_and_sample_type
13722.58cfa82de4b0c9d6adf6a4c2,58cfa82de4b0c9d6adf6a4c2,12dd02ea14d0a87df23ce3bef406fe27.filtered.,WGS,12dd02ea14d0a87df23ce3bef406fe27,12dd02ea14d0a87df23ce3bef406fe27.bam,32.75,NaN,0.08,58cfa82de4b0c9d6adf6a4c2,12dd02ea14d0a87df23ce3bef406fe27.bam,...,Duke,United States,NO,T2,N0,Infiltrating Ductal Carcinoma,Stage IIA,A21Q-09,True,Breast Invasive Carcinoma PT
13722.58cfa82de4b0c9d6adf6a502,58cfa82de4b0c9d6adf6a502,3d9f475186150ea055fddf25af7bb7e3.filtered.,WGS,3d9f475186150ea055fddf25af7bb7e3,3d9f475186150ea055fddf25af7bb7e3.bam,64.35,1.70,0.08,58cfa82de4b0c9d6adf6a502,3d9f475186150ea055fddf25af7bb7e3.bam,...,University of North Carolina,United States,NO,Not available,Not available,Endometrioid endometrial adenocarcinoma,Not available,A13L-09,False,Uterine Corpus Endometrial Carcinoma PT
13722.58cfa82de4b0c9d6adf6a4ce,58cfa82de4b0c9d6adf6a4ce,2258e57e8e0af9db6969a1da86177ca7.filtered.,WGS,2258e57e8e0af9db6969a1da86177ca7,2258e57e8e0af9db6969a1da86177ca7.bam,62.02,2.08,0.08,58cfa82de4b0c9d6adf6a4ce,2258e57e8e0af9db6969a1da86177ca7.bam,...,MSKCC,None,NO,T3,N2,Infiltrating Ductal Carcinoma,Stage IIIA,A19H-09,True,Breast Invasive Carcinoma PT
13722.58cfa82de4b0c9d6adf6a48a,58cfa82de4b0c9d6adf6a48a,142ba22e796cab1075278cd533a287c8.filtered.,WGS,142ba22e796cab1075278cd533a287c8,142ba22e796cab1075278cd533a287c8.bam,93.54,2.18,0.07,58cfa82de4b0c9d6adf6a48a,142ba22e796cab1075278cd533a287c8.bam,...,MSKCC,None,NO,Not available,Not available,Serous endometrial adenocarcinoma,Not available,A066-09,True,Uterine Corpus Endometrial Carcinoma PT
13722.58cfa82de4b0c9d6adf6a4d4,58cfa82de4b0c9d6adf6a4d4,406aecbc23505359850e57fbf05d5b67.filtered.,WGS,406aecbc23505359850e57fbf05d5b67,406aecbc23505359850e57fbf05d5b67.bam,85.32,1.85,0.08,58cfa82de4b0c9d6adf6a4d4,406aecbc23505359850e57fbf05d5b67.bam,...,MSKCC,None,NO,Not available,Not available,Serous endometrial adenocarcinoma,Not available,A066-09,True,Uterine Corpus Endometrial Carcinoma PT


# skip this section if already run

In [20]:
# import data in Q2
bt = Table(tbl_df.values.T, tbl_df.columns, tbl_df.index)
table = Artifact.import_data('FeatureTable[Frequency]', bt)

# beta div calculations
bc_dist = beta(table, 'braycurtis').distance_matrix
bc_pcoa = pcoa(bc_dist).pcoa
atch_biplot, atch_dist = rpca(table)

# save dists
bc_dist.save('../results/beta-div/bc-dist.qza')
bc_pcoa.save('../results/beta-div/bc-pcoa.qza')
atch_biplot.save('../results/beta-div/aitch-biplot.qza')
atch_dist.save('../results/beta-div/aitch-dist.qza')


/home/cmartino/.conda/envs/qiime2-2019.7/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:1575: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/cmartino/.conda/envs/qiime2-2019.7/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -4.159800441090162 and the largest is 372.571715657325.
  RuntimeWarning
/home/cmartino/.conda/envs/qiime2-2019.7/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude wit

# re-import from mem.

In [3]:
# save dists
bc_dist = Artifact.load('../results/beta-div/bc-dist.qza')
bc_pcoa = Artifact.load('../results/beta-div/bc-pcoa.qza')
atch_biplot = Artifact.load('../results/beta-div/aitch-biplot.qza')
atch_dist = Artifact.load('../results/beta-div/aitch-dist.qza')


In [4]:
# export distances
bc_dist_dm = bc_dist.view(DistanceMatrix)
atch_dist_dm = atch_dist.view(DistanceMatrix)


# permanova

In [5]:
# store results
permanova_results = {}
for dt_tmp, mf_tmp in tqdm(mf_df.groupby('disease_type')):
    if len(set(mf_tmp.sample_type)) != 2:
        continue
    # ensure matched / subset
    shared_ = set(mf_tmp.index) & set(bc_dist_dm.ids) & set(jc_dist_dm.ids) & set(atch_dist_dm.ids)
    bc_dist_dm_tmp = bc_dist_dm.copy().filter(shared_)
    jc_dist_dm_tmp = jc_dist_dm.copy().filter(shared_)
    atch_dist_dm_tmp = atch_dist_dm.copy().filter(shared_)
    # permanova within type across sample_type
    permanova_results[(dt_tmp, 'Bray-Curtis')] = permanova(bc_dist_dm_tmp,
                                                           mf_tmp['sample_type'])
    permanova_results[(dt_tmp, 'Aitchison')] = permanova(atch_dist_dm_tmp,
                                                         mf_tmp['sample_type'])


  0%|          | 0/32 [00:00<?, ?it/s]

In [6]:
# generate PERMANOVA results
permanova_results_df = pd.DataFrame(permanova_results).T
permanova_results_df.index.names = ['disease-type', 'metric']
permanova_results_df['q-value'] = permanova_results_df['p-value'] * len(set([x[0] for x in permanova_results_df.index]))
permanova_results_df.loc[permanova_results_df['q-value'] > 1, 'q-value'] = 1
permanova_results_df = permanova_results_df.loc[(slice(None), ['Bray-Curtis', 'Aitchison']), :]
permanova_results_df.to_csv('../results/beta-div/permanova-res-sample-type.csv')
permanova_results_df.to_csv('../results/tables/beta-div/permanova-res-sample-type.csv')
permanova_results_df.head(10)


method name  \
disease-type                                       metric                    
Bladder Urothelial Carcinoma                       Bray-Curtis   PERMANOVA   
                                                   Aitchison     PERMANOVA   
Breast Invasive Carcinoma                          Bray-Curtis   PERMANOVA   
                                                   Aitchison     PERMANOVA   
Cervical Squamous Cell Carcinoma and Endocervic... Bray-Curtis   PERMANOVA   
                                                   Aitchison     PERMANOVA   
Cholangiocarcinoma                                 Bray-Curtis   PERMANOVA   
                                                   Aitchison     PERMANOVA   
Colon Adenocarcinoma                               Bray-Curtis   PERMANOVA   
                                                   Aitchison     PERMANOVA   

                                                               test statistic name  \
disease-type                                       metric                            
Bladder Urothelial Carcinoma                       Bray-Curtis            pseudo-F   
                                                   Aitchison              pseudo-F   
Breast Invasive Carcinoma                          Bray-Curtis            pseudo-F   
                                                   Aitchison              pseudo-F   
Cervical Squamous Cell Carcinoma and Endocervic... Bray-Curtis            pseudo-F   
                                                   Aitchison              pseudo-F   
Cholangiocarcinoma                                 Bray-Curtis            pseudo-F   
                                                   Aitchison              pseudo-F   
Colon Adenocarcinoma                               Bray-Curtis            pseudo-F   
                                                   Aitchison              pseudo-F   

                                                               sample size  \
disease-type                                       metric                    
Bladder Urothelial Carcinoma                       Bray-Curtis         580   
                                                   Aitchison           580   
Breast Invasive Carcinoma                          Bray-Curtis        1251   
                                                   Aitchison          1251   
Cervical Squamous Cell Carcinoma and Endocervic... Bray-Curtis         364   
                                                   Aitchison           364   
Cholangiocarcinoma                                 Bray-Curtis          41   
                                                   Aitchison            41   
Colon Adenocarcinoma                               Bray-Curtis         465   
                                                   Aitchison           465   

                                                               number of groups  \
disease-type                                       metric                         
Bladder Urothelial Carcinoma                       Bray-Curtis                2   
                                                   Aitchison                  2   
Breast Invasive Carcinoma                          Bray-Curtis                2   
                                                   Aitchison                  2   
Cervical Squamous Cell Carcinoma and Endocervic... Bray-Curtis                2   
                                                   Aitchison                  2   
Cholangiocarcinoma                                 Bray-Curtis                2   
                                                   Aitchison                  2   
Colon Adenocarcinoma                               Bray-Curtis                2   
                                                   Aitchison                  2   

                                                               test statistic  \
disease-type                                       metric                     